In [1]:
import datetime
import pymongo
import os
from pymongo import MongoClient
from dotenv import load_dotenv
ruta_actual = os.getcwd() 
ruta_padre = os.path.dirname(ruta_actual)
load_dotenv('venv/.env')
import datetime
import random
import json

In [2]:
a = os.getenv('connection_url_SanManuel')
a

In [3]:
connection_url_SanManuel =  'mongodb://eefACE:9UzYm7sB5HxR4WqE3TnGpJ2MvDfXcZvA6L8O1K0I7U3N5Y8T4R6W9Q3E6T2O8H7D3E6S4L9I7T8O1L0O@192.168.100.11:27018/' 
ecommerce_main = 'Ecommerce'
eCommerce_inventario = [connection_url_SanManuel, ecommerce_main, 'inventario']
eCommerce_usuarios = [connection_url_SanManuel, ecommerce_main, 'usuario']

In [4]:
def coneccionDB(mongo_uri:str,database_name:str,collection_name:str):
    '''Permite realizar una coneccion a una colecccion en base de datos'''
    
    try:
        client = MongoClient(mongo_uri)
        db = client[database_name]
        collection = db[collection_name]
        print(f'coneccion exitosa a la coleccion: {database_name}.{collection_name}')
    except Exception as e:
        print(e)
    return collection

## Asignar un precio a los productos

In [5]:
def generar_precio_unitario():
    return random.randint(30, 200)
update = False
# Actualiza todos los documentos en la colección (nota, todos los prodcutos tienen el mismo precio)
if update:
    result = coneccionDB(*eCommerce_inventario).update_many({}, {'$set': {'precioUnitario': generar_precio_unitario()}})


## ejemplo de un usuario

In [6]:
usuario = coneccionDB(*eCommerce_usuarios).find_one({'nombre': "Daniel García"})
usuario

coneccion exitosa a la coleccion: Ecommerce.usuario


{'_id': ObjectId('65c63fbca5315cbe1489b148'),
 'usuario': 'Al92202453',
 'nombre': 'Daniel García',
 'denominacion': 'SwineSustenance Solutions Inc.',
 'password': '$2b$12$GOHmVEj0X5Wz8SglY.RvNeNVaYuCWMUuqKvx/Lut5AXIqCIpy1snK',
 'email': 'swine.sus.s@outlook.com',
 'proveedor': 1,
 'telefono': '2211847999',
 'celular': '2211847999',
 'picture': '/images/imagenes/user.png'}

## ejemplo de una  lista aleatoria de  productos

In [7]:
pipeline = [{'$sample': {'size': 3}}]
productos  = list(coneccionDB(*eCommerce_inventario).aggregate(pipeline))
productos

coneccion exitosa a la coleccion: Ecommerce.inventario


[{'_id': ObjectId('66550424b1a3e277d8fb2b65'),
  'codigoBarras': '240109GLY01',
  'fecha': datetime.datetime(2024, 5, 27, 20, 4, 4, 716000),
  'nombreOriginal': 'Yellow Nursery Gown',
  'nombre': 'Bata de guardería amarilla',
  'cajas': 100,
  'cantidadCaja': 100,
  'Total': 10000,
  'precioUnitario': 167},
 {'_id': ObjectId('66550424b1a3e277d8fb2b69'),
  'codigoBarras': '240116SSB04',
  'fecha': datetime.datetime(2024, 5, 27, 20, 16, 56, 869000),
  'nombreOriginal': 'Cholecystomy Lparoscopy Pack',
  'nombre': 'Paquete Colecistomía Lparoscopia',
  'cajas': 10,
  'cantidadCaja': 10,
  'Total': 100,
  'precioUnitario': 167},
 {'_id': ObjectId('66550424b1a3e277d8fb2b63'),
  'codigoBarras': '240109SSY01',
  'fecha': datetime.datetime(2024, 5, 27, 19, 59, 8, 330000),
  'nombreOriginal': 'Surgical Gown',
  'nombre': 'Bata quirúrgica',
  'cajas': 600,
  'cantidadCaja': 50,
  'Total': 30000,
  'precioUnitario': 167}]

In [8]:
def simularCompra(productos:list):

    for producto in productos:
        cantidad = random.randint(a=1,b=30)
        subtotal = cantidad*producto['precioUnitario']

        producto['cantidad'] = cantidad
        producto['subtotal'] = subtotal
    
    return productos
        

In [21]:
firstkeys = ['codigoBarras', 'nombre', 'precioUnitario']
productos = list(map(lambda d: {k: d[k] for k in firstkeys}, productos))
productos = simularCompra(productos=productos)
productos

[{'codigoBarras': '240109GLY01',
  'nombre': 'Bata de guardería amarilla',
  'precioUnitario': 167,
  'cantidad': 21,
  'subtotal': 3507},
 {'codigoBarras': '240116SSB04',
  'nombre': 'Paquete Colecistomía Lparoscopia',
  'precioUnitario': 167,
  'cantidad': 19,
  'subtotal': 3173},
 {'codigoBarras': '240109SSY01',
  'nombre': 'Bata quirúrgica',
  'precioUnitario': 167,
  'cantidad': 17,
  'subtotal': 2839}]

## obtener totales

In [10]:
totalCompra    =  sum([producto['subtotal'] for producto in productos])
totalArticulos =  sum([producto['cantidad'] for producto in productos])

## crear un registro de un ticket


In [24]:
def crearTicket():
    registro = dict()

    usuario = coneccionDB(*eCommerce_usuarios).find_one({'nombre': "Daniel García"})
    registro['idUser'] = usuario['usuario']
    registro['userName'] = usuario['nombre']
    #
    sale_size = random.randint(a=1, b= 10)
    pipeline = [{'$sample': {'size': sale_size}}]
    productos  = list(coneccionDB(*eCommerce_inventario).aggregate(pipeline))

    firstkeys = ['codigoBarras', 'nombre', 'precioUnitario']
    productos = list(map(lambda d: {k: d[k] for k in firstkeys}, productos))
    productos = simularCompra(productos=productos)
    
    registro['compra'] = productos
    #
    registro['totalCompra']    =  sum([producto['subtotal'] for producto in productos])
    registro['totalArticulos'] =  sum([producto['cantidad'] for producto in productos])
    registro['metidoPago']     =  random.choice(['Efectivo', 'PayPal', 'Tarjeta'])

    return registro

In [25]:
ticket = crearTicket()
ticket

coneccion exitosa a la coleccion: Ecommerce.usuario
coneccion exitosa a la coleccion: Ecommerce.inventario


{'idUser': 'Al92202453',
 'userName': 'Daniel García',
 'compra': [{'codigoBarras': '240109GLY01',
   'nombre': 'Bata de guardería amarilla',
   'precioUnitario': 167,
   'cantidad': 4,
   'subtotal': 668},
  {'codigoBarras': '240116SSB03',
   'nombre': 'Paquete ortopédico general',
   'precioUnitario': 167,
   'cantidad': 13,
   'subtotal': 2171},
  {'codigoBarras': '240109SSY01',
   'nombre': 'Bata quirúrgica',
   'precioUnitario': 167,
   'cantidad': 8,
   'subtotal': 1336},
  {'codigoBarras': '240116SSB05',
   'nombre': 'Paquete de cistocopia',
   'precioUnitario': 167,
   'cantidad': 26,
   'subtotal': 4342},
  {'codigoBarras': '240116SSB01',
   'nombre': 'Paquete Universal',
   'precioUnitario': 167,
   'cantidad': 25,
   'subtotal': 4175}],
 'totalCompra': 12692,
 'totalArticulos': 76,
 'metidoPago': 'Tarjeta'}

In [27]:
def EvaluarCapacidad():
    productos  = list(coneccionDB(*eCommerce_inventario).find({"codigoBarras":{"$in":["240116SSB04"]}}))
    productos[0]['Total']

coneccion exitosa a la coleccion: Ecommerce.inventario


[{'_id': ObjectId('66550424b1a3e277d8fb2b69'),
  'codigoBarras': '240116SSB04',
  'fecha': datetime.datetime(2024, 5, 27, 20, 16, 56, 869000),
  'nombreOriginal': 'Cholecystomy Lparoscopy Pack',
  'nombre': 'Paquete Colecistomía Lparoscopia',
  'cajas': 10,
  'cantidadCaja': 10,
  'Total': 100,
  'precioUnitario': 167}]

##  Funcion para obtener un producto por nombre

In [ ]:
def getProduct(product_name:str):
    """ 
    obtener la informacion 
    """
    eCommerce_inventario = [connection_url_SanManuel, ecommerce_main, 'inventario']
    product = coneccionDB(*eCommerce_inventario).find_one({'nombre': product_name})

    return product

